# prelim

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

# depending on your IDE, you might need to add datathon_eth. in front of data
from data import DataLoader, SimpleEncoding

# depending on your IDE, you might need to add datathon_eth. in front of forecast_models
from forecast_models import SimpleModel

country = "IT" 
"""

Train and evaluate the models for IT and ES

"""

# Inputs
input_path = r"datasets2025"
output_path = r"outputs"

# Load Datasets
loader = DataLoader(input_path)
# features are holidays and temperature
training_set, features, example_results = loader.load_data(country)

In [ ]:
training_set

In [ ]:
training_set[training_set.columns[4]][10000:10200].plot()

In [ ]:
class SimpleEncoding:
    """
    This class is an example of dataset encoding.

    """

    def __init__(
        self,
        consumption: pd.Series,
        features: pd.Series,
        end_training,
        start_forecast,
        end_forecast,
    ):
        self.consumption_mask = ~consumption.isna()
        self.consumption = consumption[self.consumption_mask]
        self.features = features
        self.end_training = end_training
        self.start_forecast = start_forecast
        self.end_forecast = end_forecast

    def meta_encoding(self):
        """
        This function returns the feature, split between past (for training) and future (for forecasting)),
        as well as the consumption, without missing values.
        :return: three numpy arrays

        """
        features_past = self.features[: self.end_training].values.reshape(-1, 1)
        features_future = self.features[
            self.start_forecast : self.end_forecast
        ].values.reshape(-1, 1)

        features_past = features_past[self.consumption_mask]

        return features_past, features_future, self.consumption

In [ ]:
training_set.isna().sum()

In [ ]:
training_set[training_set.columns[0]].plot()

In [ ]:
training_set[training_set.columns[0]].plot()

In [ ]:
import pandas as pd
import numpy as np

class Encoder:
    """
    This class is an example of dataset encoding for a multivariate feature set.
    """

    def __init__(
        self,
        consumption: pd.Series,
        features: pd.DataFrame,
        end_training,
        start_forecast,
        end_forecast,
    ):
        # assert isinstance(features, pd.DataFrame), "features must be a DataFrame"
        self.consumption_mask = ~consumption.isna()
        self.consumption = consumption[self.consumption_mask]
        self.features = features
        self.end_training = end_training
        self.start_forecast = start_forecast
        self.end_forecast = end_forecast

    def meta_encoding(self):
        """
        This function returns the feature matrix split between past (for training)
        and future (for forecasting), as well as the non-missing consumption values.
        :return: (features_past, features_future, consumption) as numpy arrays
        """
        # Select past and future feature slices
        features_past = self.features.loc[:self.end_training].values
        features_future = self.features.loc[self.start_forecast:self.end_forecast].values

        # Apply the consumption mask to past features
        features_past = features_past[self.consumption_mask]

        return features_past, features_future, self.consumption.values


In [ ]:
input_path = r"datasets2025"
output_path = r"outputs"

# Load Datasets
loader = DataLoader(input_path)
# features are holidays and temperature
zone = "IT" # "ES"
training_set, features, example_results = loader.load_data(zone)
start_training = training_set.index.min()
end_training = training_set.index.max()
start_forecast, end_forecast = example_results.index[0], example_results.index[-1]
consumption = training_set

feature_dummy = features['temp'].loc[start_training:]

# encoding = Encoder(
#     consumption, feature_dummy, end_training, start_forecast, end_forecast
# )

In [ ]:
training_set.iloc[:, 0:5].pct_change().corr()

In [ ]:
training_set.iloc[-2900:-2700, 0:5].plot()

In [ ]:
# rollout_it = pd.read_csv("datasets2025/rollout_data_IT.csv", index_col = 0, parse_dates = True)
# rollout_it.head()

In [ ]:
# customer_id = 1
# joint_df = pd.concat([rollout_it.loc[:, rollout_it.columns[customer_id]], 
#                     training_set.loc[:, training_set.columns[customer_id]]],
#                     axis = 1)
# joint_df

In [ ]:
# joint_df.iloc[10800:11000].plot()

# Some ML

In [ ]:
training_set

## Fourier Encode

In [ ]:
customer = training_set.columns[0]
X = training_set[customer]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from lightgbm import LGBMRegressor

# --------- Model Classes ---------

class BaseModel:
    def train(self, X, y):
        raise NotImplementedError

    def predict(self, X):
        raise NotImplementedError

class LightGBMModel(BaseModel):
    def __init__(self, **params):
        self.model = LGBMRegressor(**params)

    def train(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

class LinearRegressionModel(BaseModel):
    def __init__(self, **params):
        self.model = LinearRegression(**params)

    def train(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

# --------- Time Feature Extractor ---------

class TimeFeaturesExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Expecting X to be a DataFrame with a DatetimeIndex.
        df = pd.DataFrame(index=X.index)
        df["hour"] = X.index.hour
        df["dayofweek"] = X.index.dayofweek
        return df

# --------- Main Forecast Function ---------

def forecast_time_series(
    X: pd.Series,
    forecast_horizon: int = 720,  # forecast 720 time steps which are the test set
    model_type: str = "lightgbm",
    **model_params
):
    # Ensure the index is a DatetimeIndex and drop missing values
    if not isinstance(X.index, pd.DatetimeIndex):
        raise ValueError("The time series index must be a pandas DatetimeIndex.")
    X = X.dropna()

    # Split data into training and test sets
    if len(X) < forecast_horizon:
        raise ValueError("The time series does not have enough data for the specified forecast horizon.")
        
    train_size = len(X) - forecast_horizon
    X_train_raw = X.iloc[:train_size]
    X_test_raw = X.iloc[train_size:]
    
    y_train = X_train_raw.values
    y_test = X_test_raw.values

    # Feature pipeline: extract time features and then one-hot encode them.
    feature_extractor = Pipeline([
        ('time_features', TimeFeaturesExtractor()),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Fit the feature extractor on the training data and transform both sets
    X_train_features = feature_extractor.fit_transform(X_train_raw.to_frame())
    X_test_features = feature_extractor.transform(X_test_raw.to_frame())

    # Choose the forecasting model
    model_classes = {
        "lightgbm": LightGBMModel,
        "linear": LinearRegressionModel,
    }
    if model_type not in model_classes:
        raise ValueError(f"Unsupported model type: {model_type}. Choose from {list(model_classes.keys())}")

    model = model_classes[model_type](**model_params)
    model.train(X_train_features, y_train)

    # Predict the test set in one go
    y_pred = model.predict(X_test_features)

    # Calculate and print Mean Squared Error for the test set
    mse = np.mean(np.square(y_pred - y_test))
    print("Test MSE:", mse)

    # Plot ground truth vs. predicted values for the 720-step test set
    plt.figure(figsize=(12, 6))
    plt.plot(X_test_raw.index, y_test, label='Ground Truth', linewidth=2)
    plt.plot(X_test_raw.index, y_pred, label='Predicted', linestyle='--')
    plt.title(f"{forecast_horizon}-Step Forecast Comparison ({model_type})")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Optionally, return a DataFrame with the ground truth and predicted values
    results = pd.DataFrame({
        "Ground Truth": y_test,
        "Predicted": y_pred
    }, index=X_test_raw.index)
    
    return results

In [ ]:
forecast_time_series(X, 720, model_type="lightgbm")

In [ ]:
forecast_time_series(X, 720, model_type="linear")

## RBF Encode

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from lightgbm import LGBMRegressor


# --------- Model Classes ---------

class BaseModel:
    def train(self, X, y):
        raise NotImplementedError

    def predict(self, X):
        raise NotImplementedError


class LightGBMModel(BaseModel):
    def __init__(self, **params):
        self.model = LGBMRegressor(**params)

    def train(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)


class LinearRegressionModel(BaseModel):
    def __init__(self, **params):
        self.model = LinearRegression(**params)

    def train(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)


# --------- Time Feature Extractor ---------

class TimeFeaturesExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = pd.DataFrame(index=X.index)
        df["hour"] = X.index.hour
        df["dayofweek"] = X.index.dayofweek
        return df


# --------- Main Forecast Function ---------

def forecast_time_series(
    X: pd.Series,
    forecast_horizon: int = 240,
    test_frac: float = 0.2,
    model_type: str = "lightgbm",
    **model_params
):
    # Drop missing values
    X = X.dropna()

    # Ensure enough data for test
    total_len = len(X)
    test_size = max(int(total_len * test_frac), forecast_horizon)
    train_size = total_len - test_size

    X_train_raw = X.iloc[:train_size]
    X_test_raw = X.iloc[train_size:train_size + forecast_horizon]
    y_train = X_train_raw.values
    y_test = X_test_raw.values

    # Feature pipeline
    feature_extractor = Pipeline([
        ('time_features', TimeFeaturesExtractor()),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    X_train_features = feature_extractor.fit_transform(X_train_raw.to_frame())
    X_test_features = feature_extractor.transform(X_test_raw.to_frame())

    # Choose model
    model_classes = {
        "lightgbm": LightGBMModel,
        "linear": LinearRegressionModel,
    }

    if model_type not in model_classes:
        raise ValueError(f"Unsupported model type: {model_type}. Choose from {list(model_classes.keys())}")

    model = model_classes[model_type](**model_params)
    model.train(X_train_features, y_train)
    y_pred = model.predict(X_test_features)

    print("MSE:", np.mean(np.square(y_pred - y_test)))

    # Plot predictions
    plt.figure(figsize=(12, 6))
    plt.plot(X_test_raw.index, y_test, label='Ground Truth', linewidth=2)
    plt.plot(X_test_raw.index, y_pred, label='Forecast', linestyle='--')
    plt.title(f"{forecast_horizon}-Step Forecast ({model_type})")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# LightGBM
forecast_time_series(X, model_type="lightgbm", forecast_horizon=720, n_estimators=100, learning_rate=0.05)

In [ ]:
forecast_time_series(X, model_type="linear", forecast_horizon=720)

# Baseline Prediction

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

# depending on your IDE, you might need to add datathon_eth. in front of data
from data import DataLoader, SimpleEncoding

# depending on your IDE, you might need to add datathon_eth. in front of forecast_models
from forecast_models import SimpleModel

country = "ES" 
"""

Train and evaluate the models for IT and ES

"""

# Inputs
input_path = r"datasets2025"
output_path = r"outputs"

# Load Datasets
loader = DataLoader(input_path)
# features are holidays and temperature
training_set, features, example_results = loader.load_data(country)
rollout, holidays = loader.load_additional_data(country)

In [ ]:
from data import DataLoader, DatasetEncoding

team_name = "HANGUK_ML"
# Data Manipulation and Training
start_training = training_set.index.min()
end_training = training_set.index.max()
start_forecast, end_forecast = example_results.index[0], example_results.index[-1]

range_forecast = pd.date_range(start=start_forecast, end=end_forecast, freq="1H")

de = DatasetEncoding(
    consumption=training_set,
    features=features,
    rollout=rollout,
    holiday=holidays,
    end_training=end_training,
    start_forecast=start_forecast,
    end_forecast=end_forecast,
)
df = de.generate_time_series_features(
    start_time=start_training,
    end_time=end_training,
)

In [ ]:
df = de.generate_dataset(customer_id=1, window_size=24 * 7, forecast_skip=1, forecast_horizon=7)
df_train = df[:-720]
df_test = df[-720:]

## naive1

In [ ]:
from baseline_models import naive

model_naive = naive(prediction_window=720)
pred = model_naive.predict(df_train)
pred.plot()

In [ ]:
joint_df = pd.concat([df_test["consumption"], pred], axis = 1)
print("MSE = ", np.mean(np.square(df_test["consumption"] - pred)))
joint_df.plot()

## naive2

In [ ]:
# from baseline_models import naive2

# model_naive = naive2(prediction_window=737)
# model_naive.train(df_train)
# pred = model_naive.predict(df_train)
# pred.plot()

In [ ]:
# joint_df = pd.concat([df_test["consumption"], pred], axis = 1)
# print("MSE = ", np.mean(np.square(df_test["consumption"] - pred)))
# joint_df.plot()

In [ ]:
# joint_df = pd.concat([df_test["consumption"], pred], axis = 1)
# print("MSE = ", np.mean(np.square(df_test["consumption"] - pred)))
# joint_df.plot()

## naive_3

In [ ]:
class plain:

    def __init__(self, prediction_window: int = 720):
        # prediction_window is the number of future hourly time steps to forecast.
        self.prediction_window = prediction_window

    def train(self, **kwargs):
        # No training is needed for a naive predictor.
        pass

    def predict(self, x: pd.Series) -> pd.Series:
        """
        Predict the next prediction_window timesteps after the last index in x.

        The procedure is:
          - Define a prediction range: hourly timestamps starting from the next midnight after
            the last observation.
          - Candidate one: lookup the corresponding period one year ago.
          - Candidate two: lookup the corresponding period one month ago.
          - For each timestep in the prediction range, if both candidate values are available,
            take their average; if only one is available, use that value; if both are missing,
            return 0.
        
        Parameters:
          x (pd.Series): Time series with a DatetimeIndex.
        
        Returns:
          pd.Series: Forecasted values for the next prediction_window hours, with the future timestamps.
        """
        # Ensure x has a DatetimeIndex
        if not isinstance(x.index, pd.DatetimeIndex):
            raise ValueError("Input time series must have a DatetimeIndex.")

        x = x['consumption']

        # Determine the current observation (last timestamp)
        current_observation = x.index[-1]
        # Determine the next day’s midnight:
        prediction_start = (current_observation + pd.Timedelta(days=1)).normalize()
        # Create the prediction range with hourly frequency
        prediction_range = pd.date_range(start=prediction_start, periods=self.prediction_window, freq='H')

        candidate_range_year = prediction_range - DateOffset(years=1)
        candidate_range_month = prediction_range - DateOffset(months=1)

        candidate_one = x.reindex(candidate_range_year)
        candidate_one.index = prediction_range  # Carry candidate values into the future index

        candidate_two = x.reindex(candidate_range_month)
        candidate_two.index = prediction_range  # Carry candidate values into the future index
        
        df_candidates = pd.DataFrame({
            "candidate_one": candidate_one,
            "candidate_two": candidate_two
        }, index=prediction_range)

        forecast = df_candidates.mean(axis=1, skipna=True)
        forecast = forecast.fillna(0)

        # Debug prints (optional)
        print("Current observation:", current_observation)
        print("Prediction range starts at:", prediction_range[0])
        print("Candidate range (1 year ago) starts at:", candidate_range_year[0])
        print("Candidate range (1 month ago) starts at:", candidate_range_month[0])

        return forecast

## Clustering

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

# depending on your IDE, you might need to add datathon_eth. in front of data
from data import DataLoader, SimpleEncoding

# depending on your IDE, you might need to add datathon_eth. in front of forecast_models
from forecast_models import SimpleModel

country = "IT" 
"""

Train and evaluate the models for IT and ES

"""

# Inputs
input_path = r"datasets2025"
output_path = r"outputs"

# Load Datasets
loader = DataLoader(input_path)
# features are holidays and temperature
training_set, features, example_results = loader.load_data(country)
rollout, holidays = loader.load_additional_data(country)

In [ ]:
training_set

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Assuming 'training_set' is your DataFrame with a datetime index
# and each column corresponds to a different customer.

# Step 1: Compute basic features for each customer

# Basic statistics computed on the raw time series
features = pd.DataFrame(index=training_set.columns)
features['mean'] = training_set.mean()
features['std'] = training_set.std()
features['max'] = training_set.max()
features['min'] = training_set.min()

# Optionally, resample to daily frequency to extract daily consumption features.
daily_data = training_set.resample('D').mean()

# Compute daily-based features
features['daily_mean'] = daily_data.mean()
features['daily_std'] = daily_data.std()

# You can further compute additional features such as:
# - The percentage of missing values per customer
# - Autocorrelation coefficients (e.g., lag-1 autocorrelation)
features['missing_rate'] = training_set.isna().mean()

# Example: Calculate lag-1 autocorrelation for each customer
def lag1_autocorr(x):
    x = x.dropna()  # drop NaN values for autocorrelation calculation
    if len(x) < 2:
        return np.nan
    return x.autocorr(lag=1)

features['lag1_autocorr'] = training_set.apply(lag1_autocorr)

print("Extracted Features:")
print(features.head())

# Step 2: Preprocess the feature set

# Fill any remaining NaN values that might be present after feature extraction
features = features.fillna(features.mean())

# Standardize the features (important for many clustering algorithms)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Step 3: Apply clustering

# Here we choose k-means as an example. You may adjust the number of clusters as needed.
n_clusters = 3  # You can choose the number of clusters based on domain knowledge or via methods like the elbow method.
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(features_scaled)

# Add the cluster labels to your features DataFrame
features['cluster'] = clusters

print("\nCluster assignments:")
print(features[['cluster']].head())

# Optional: Visualize the clustering using the first two principal components.
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

plt.figure(figsize=(8, 6))
plt.scatter(features_pca[:, 0], features_pca[:, 1], c=clusters, cmap='viridis', edgecolor='k', s=100)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Clustering of Customers based on Time-Series Features')
plt.colorbar(label='Cluster')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Suppose 'training_set' is your pandas DataFrame with a datetime index
# and each column corresponds to a customer’s power consumption time series.

# Step 1: Fill missing values
# Here we fill forward then backward; adjust if a different imputation is preferred.
df_filled = training_set.fillna(method='ffill').fillna(method='bfill')

# Step 2: Choose a reference time series.
# For instance, here we select the first customer as the reference.
ref_customer = df_filled.columns[0]
ref_series = df_filled[ref_customer]

# Step 3: Compute the Euclidean distance between the reference and each other customer.
# This distance is computed over all time points.
distances = {}
for customer in df_filled.columns:
    if customer == ref_customer:
        continue  # Skip comparing the reference with itself.
    # Calculate Euclidean distance
    distances[customer] = np.linalg.norm(ref_series - df_filled[customer])

# Step 4: Select the four customers that are closest to the reference.
closest_customers = sorted(distances, key=distances.get)[:4]

# Include the reference customer in the list of most similar paths.
selected_customers = [ref_customer] + closest_customers

print("Selected customers:", selected_customers)

# Step 5: Plot the selected five time series together.
df_filled = df_filled[-200:]
plt.figure(figsize=(12, 6))
for customer in selected_customers:
    plt.plot(df_filled.index, df_filled[customer], label=customer)
plt.xlabel("Time")
plt.ylabel("Power Consumption")
plt.title("Five Most Similar Customer Paths")
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import esig as ts

# Assuming training_set is a DataFrame with datetime index and customer columns
# Transpose: now each row is a time series of a customer
training_set = training_set[training_set.columns[:300]]
training_set = training_set[-10_000:].fillna(0)
customer_series = training_set.T


def compute_signatures(series_df, depth=2):
    """
    Compute the signature of each customer's time series.
    Each row of series_df is a time series for one customer.
    """
    signatures = []
    for _, path in series_df.iterrows():
        # Convert to array and reshape (T, 1) to be a 1D path
        path_array = path.to_numpy().reshape(-1, 1)
        sig = ts.stream2sig(path_array, depth)
        signatures.append(sig)
    return np.vstack(signatures)

sig_features = compute_signatures(customer_series, depth=10)

# Compute the kernel matrix (Gram matrix)
kernel_matrix = np.dot(sig_features, sig_features.T)


In [ ]:
from sklearn.decomposition import KernelPCA

# 4. Run Kernel PCA with 2 components
kpca = KernelPCA(kernel='precomputed', n_components=2)
X_kpca = kpca.fit_transform(kernel_matrix)

# 5. Plot
plt.figure(figsize=(8, 6))
plt.scatter(X_kpca[:, 0], X_kpca[:, 1], c='steelblue', edgecolor='k', s=80)
plt.title("Kernel PCA (Signature Kernel) - 2 Components")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.grid(True)
plt.tight_layout()
plt.show()

## naive improved

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

class TimeOLSmodel:
    def __init__(self, prediction_window: int = 720):
        self.prediction_window = prediction_window
        self.model = LinearRegression()
        self.encoder = OneHotEncoder(handle_unknown='ignore')

    def _generate_time_features(self, timestamps: pd.Series):
        """
        Generate one-hot encoded hour of day and day of week features from timestamps.
        """
        df = pd.DataFrame()
        df["hour"] = timestamps.dt.hour
        df["dayofweek"] = timestamps.dt.dayofweek
        return self.encoder.transform(df)

    def train(self, x: pd.DataFrame, y: np.ndarray):
        """
        x should include a 'timestamp' (datetime64) column and a 'consumption' column.
        """
        if not pd.api.types.is_datetime64_any_dtype(x["timestamp"]):
            x["timestamp"] = pd.to_datetime(x["timestamp"])

        # Fit the encoder on the time features
        df = pd.DataFrame()
        df["hour"] = x["timestamp"].dt.hour
        df["dayofweek"] = x["timestamp"].dt.dayofweek
        self.encoder.fit(df)

        time_features = self._generate_time_features(x["timestamp"])
        
        # Combine with past consumption as feature
        features = np.column_stack([x["consumption"].values.reshape(-1, 1), time_features.values])

        self.model.fit(features, y)

    def predict(self, x: pd.DataFrame):
        """
        x should include a 'timestamp' (datetime64) column and a 'consumption' column.
        """
        if not pd.api.types.is_datetime64_any_dtype(x["timestamp"]):
            x["timestamp"] = pd.to_datetime(x["timestamp"])

        time_features = self._generate_time_features(x["timestamp"])
        features = np.column_stack([x["consumption"].values.reshape(-1, 1), time_features.values])

        return self.model.predict(features)